In [9]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.cbook as cbook


columns = 'columns'
df1_mode =  lambda x:x.value_counts().index[0]


# ---- IMPORT JSON FILES INTO DATAFRAMES ----- #

# Read json file 
df1 = pd.read_json ('/Users/bexa/Documents/Computer Science MSc/Advanced Programming/Summative/facilities.json')

# create new YEAR column

# ---- TEST SECTION ----- #

# facility:FA0019645, zip code: 25719. total violations: 4
# print("TEST RESULTS: ", df1.loc[df1['FACILITY ID'] == 'FA0260258' ])



# ---- WRANGLING FOR VIOLATION CODE COUNT SUBSET ----- #

# total violations PER facility
total_score = df1['VIOLATION CODE'].value_counts().reset_index() # reset index
total_score.columns = ['VIOLATION CODE', 'TOTAL VIOLATIONS']
total_score = total_score.sort_values(by=['VIOLATION CODE'])

# Remove values below this total amount
values_below = 0
# Drop rows, keep rest 
total_score = total_score[total_score['TOTAL VIOLATIONS'] > values_below]


# ---- WRANGLING TOTAL VIOLATION PER FACILITY COMPARE AGAINST ZIP CODES ----- #

# Grab df5_merge and count up violations per facility 
df6_count = df1['FACILITY ID'].value_counts().reset_index()

# Total violations of each facility
df6_count.columns = ['FACILITY ID', 'TOTAL VIOLATIONS']

# Add total violations to end of facilities dataframe. 
df6_merge = pd.merge(df1, df6_count, how='inner', on='FACILITY ID')

# Groups by zip then shows total violations per facility grouped via zip code
df6_total = df6_merge.groupby('ZIP CODES')['FACILITY ID'].value_counts().to_frame(name = 'TOTAL VIOLATIONS').reset_index()

# std calculates variability and avg distance from mean
agg_group = [ ('FACILITIES', 'count'), ('VIOLATIONS', 'sum'), ('STD','std'), ('MEAN','mean'),('MEDIAN','median'), ('MODE', df1_mode), ('MIN', 'min'), ('MAX', 'max')]
df6_group = df6_total.groupby('ZIP CODES')['TOTAL VIOLATIONS'].agg(agg_group)
df6_group['RANGE'] = df6_group['MAX'] - df6_group['MIN']

# Fill in empty or NaN values as 0
df6_group = df6_group.fillna(0)

# sort by total violations largest to smallest 
# df6 = df6_group.sort_values(by=['FACILITIES'], ascending=False)

# Resets index and ZIP CODES is now level 0 column header
df6 = df6_group.reset_index()

# Turn zip codes into a string 
df6_str = df6['ZIP CODES'].astype(str)

# Random colours
N = df6.shape[0]
colours = np.random.rand(N)/2

# DF6 is the final cleaned, prepped, wrangled DF to use for graphs #


In [10]:
print(df6)

     ZIP CODES  FACILITIES  VIOLATIONS       STD      MEAN  MEDIAN  MODE  MIN  \
0         2443          51         257  4.363305  5.039216     4.0     3    1   
1         2445         113         576  3.999921  5.097345     5.0     3    1   
2         2447         247        1204  3.760681  4.874494     4.0     1    1   
3         2451          52         265  2.651375  5.096154     6.0     7    1   
4         2832          87         486  4.945168  5.586207     4.0     3    1   
..         ...         ...         ...       ...       ...     ...   ...  ...   
277      27323         164         703  3.447708  4.286585     4.0     2    1   
278      27324         185         815  2.800034  4.405405     5.0     5    1   
279      27325          67         268  3.584478  4.000000     4.0     4    1   
280      27326           5          20  1.581139  4.000000     4.0     2    2   
281      27334          15          51  1.404076  3.400000     4.0     4    1   

     MAX  RANGE  
0     25 